In [27]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

In [38]:
pbp = nfl.import_pbp_data([2024])
weekly_roster_data = nfl.import_weekly_rosters([2024])
earnings_data = pd.read_csv("overthecap/overthecap-earnings.csv")

2024 done.
Downcasting floats.


In [39]:
wr_data = weekly_roster_data[['team', 'week', 'player_name', 'player_id', 'position', 'status']]

wr_data_with_earnings = pd.merge(
    wr_data,
    earnings_data,
    left_on='player_name',
    right_on='player_name',
    how='left'
)

# Fix: use .str.contains instead of .contains, and handle possible NaNs
wr_data_with_earnings[wr_data_with_earnings['player_name'].str.contains("Chauncey Gardner", na=False)]

,team_x,week,player_name,player_id,position,status,year,team_y,base_salary,prorated_bonus,Unnamed: 5,guaranteed_salary,capnumber,cap_percent,cash_paid
